In [1]:
import sys
sys.path.insert(0,"/home/unibas/boittier/fdcm_python/PyDCM-1")
# Basics
import os
import numpy as np

# Load FDCM modules
from pydcm import Scan, DCM, mdcm
# Optimization
from scipy.optimize import minimize
import pickle
import pandas as pd


In [2]:
def get_clcl(local_pos, charges):
    NCHARGES = len(charges)
    _clcl_ = np.zeros(NCHARGES)
    for i in range(NCHARGES):
        if (i+1) % 4 == 0:
            _clcl_[i] = charges[i]
        else:
            _clcl_[i] = local_pos[i - ((i)//4)]
    return _clcl_

def set_bounds(local_pos, change=0.1):
    bounds = []
    for i, x in enumerate(local_pos):
        bounds.append((x - abs(x)*change, x + abs(x)*change))
    return tuple(bounds)

# Define simple charge constrained function returning RMSE for given local MDCM
# configuration
local_ref = None
def mdcm_rmse(local_pos, local_ref=local_ref, l2=3.):
    _clcl_ = get_clcl(local_pos, charges)
    mdcm.set_clcl(_clcl_)
    rmse = mdcm.get_rmse()
    if local_ref is not None:
        l2diff = l2*np.sum((local_pos - local_ref)**2)/local_pos.shape[0]
        print(rmse, l2diff)
        rmse += l2diff 
    return rmse



In [23]:
mdcm_cxyz = "/home/unibas/boittier/fdcm_project/mdcms/methanol/10charges.xyz"
mdcm_clcl = "/home/unibas/boittier/MDCM/examples/multi-conformer/5-charmm-files/10charges.dcm"

def optimize_mdcm(nodes_to_average, first=True, local_pos=None):

    scan_fesp = []
    scan_fdns = []
    
    if first==True:
        # Prepare some cube file list
        scan_fesp = [
            "/home/unibas/boittier/MDCM/examples/multi-conformer/ref/scan0.p.cube"
        ]
        scan_fdns = [
            "/home/unibas/boittier/MDCM/examples/multi-conformer/ref/scan0.d.cube"
        ]


    for node in nodes_to_average:
        scan_fesp.append(f"/data/unibas/boittier/graphscan/methanol/t3/p{node}.p.cube")
        scan_fdns.append(f"/data/unibas/boittier/graphscan/methanol/t3/p{node}.d.cube")

    Nfiles = len(scan_fesp)
    Nchars = int(np.max([
        len(filename) for filelist in [scan_fesp, scan_fdns] 
        for filename in filelist]))

    esplist = np.empty([Nfiles, Nchars], dtype='c')
    dnslist = np.empty([Nfiles, Nchars], dtype='c')

    for ifle in range(Nfiles):
        esplist[ifle] = "{0:{1}s}".format(scan_fesp[ifle], Nchars)
        dnslist[ifle] = "{0:{1}s}".format(scan_fdns[ifle], Nchars)

    # Load cube files, read MDCM global and local files
    mdcm.load_cube_files(Nfiles, Nchars, esplist.T, dnslist.T)
    mdcm.load_clcl_file(mdcm_clcl)
    mdcm.load_cxyz_file(mdcm_cxyz)

    # Write MDCM global from local and Fitted ESP cube files
    mdcm.write_cxyz_files()
    mdcm.write_mdcm_cube_files()

    # Get and set local MDCM array (to check if manipulation is possible)
    clcl = mdcm.mdcm_clcl
    mdcm.set_clcl(clcl)
    clcl = mdcm.mdcm_clcl
    
    if local_pos is not None:
        mdcm.set_clcl(local_pos)
        clcl = mdcm.mdcm_clcl
        
    # print(clcl)

    # Get and set global MDCM array (to check if manipulation is possible)
    cxyz = mdcm.mdcm_cxyz
    mdcm.set_cxyz(cxyz)
    cxyz = mdcm.mdcm_cxyz

    # Get RMSE, averaged or weighted over ESP files, or per ESP file each
    rmse = mdcm.get_rmse()
    print(rmse)
    srmse = mdcm.get_rmse_each(Nfiles)
    print(srmse)

    #  save an array containing original charges
    charges = clcl.copy()
    local_pos = clcl[np.mod(np.arange(clcl.size)+1, 4) != 0]

    local_ref = local_pos.copy()

    def mdcm_rmse(local_pos, local_ref=local_ref, l2=10.):
        _clcl_ = get_clcl(local_pos, charges)
        mdcm.set_clcl(_clcl_)
        rmse = mdcm.get_rmse()
        if local_ref is not None:
            l2diff = l2*np.sum((local_pos - local_ref)**2)/local_pos.shape[0]
            # print(rmse, l2diff)
            rmse += l2diff 
        return rmse


    # bounds = set_bounds(local_pos, change=0.5)
    bounds = None

    # Apply simple minimization without any feasibility check (!)
    # Leads to high amplitudes of MDCM charges and local positions
    res = minimize(
        mdcm_rmse, local_pos,
        method="L-BFGS-B",
        options={'disp': None, 'maxls': 20, 'iprint': -1, 'gtol': 1e-06, 
                 'eps': 1e-09, 'maxiter': 15000, 
                 'ftol': 1e-8, 'maxcor': 10, 'maxfun': 15000})
    print(res)

    # Recompute final RMSE each
    srmse = mdcm.get_rmse_each(Nfiles)
    print(srmse)

    mdcm.write_cxyz_files()

    #  get the local charges array after optimization
    clcl_out = get_clcl(res.x, charges)

    # print(clcl_out)
    difference = np.sum((res.x - local_ref)**2)/local_pos.shape[0]
    print("charge RMSD:", difference)
    
    start_node = nodes_to_average[0]
    obj_name = f"pickles/{start_node}_clcl.obj"
    
    #  save as pickle
    filehandler = open(obj_name,"wb")
    pickle.dump(clcl_out, filehandler)

    # Not necessary but who knows when it become important to deallocate all 
    # global arrays
    mdcm.dealloc_all()


In [24]:
nodes_to_average = [1, 335, 1481, 1864, 1906, 2081, 2418]

optimize_mdcm(nodes_to_average, first=True, local_pos=None)

0.9410256904903156
[0.38855791 1.07333924 1.34799029 0.8038131  0.83496734 0.9357169
 0.84886047 1.0096992 ]
      fun: 0.6761533458506446
 hess_inv: <30x30 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.60458329e-04,  7.18314297e-05,  2.97428748e-04,  1.63402625e-03,
        6.50590693e-05, -1.56985536e-03, -2.59170456e-03, -5.24469343e-04,
        3.26272343e-03, -2.69140258e-03,  2.84217087e-05,  3.43003403e-03,
        1.97242228e-03,  3.38729045e-04, -1.96731515e-03,  2.13939977e-03,
        5.07593967e-04, -3.14592788e-03,  6.30606678e-04, -1.26565425e-04,
       -1.28008715e-04, -3.46167539e-04, -4.55191440e-06,  2.69007039e-04,
        6.67244038e-05,  2.65232281e-04, -4.46309656e-05,  9.25926003e-05,
        3.64708264e-04,  1.31006317e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 2945
      nit: 78
     njev: 95
   status: 0
  success: True
        x: array([ 0.50155069, -0.0067018 ,  0.046156  ,  0.0287527 , -0.01510663,
        

In [25]:
nodes_to_average = [2418, 1, 1058, 1520, 2004, 2384]
local_pos_saved = pd.read_pickle("pickles/1_clcl.obj")
optimize_mdcm(nodes_to_average, first=False, local_pos=local_pos_saved)

0.7117024897015288
[0.58418639 0.59628131 0.81955531 0.71203565 0.8930223  0.60693321]
      fun: 0.6589985531653443
 hess_inv: <30x30 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.08246742e-04,  1.26010313e-04,  3.95239397e-05,  9.51461133e-05,
       -5.24025268e-04, -5.49560397e-05,  3.36730634e-04,  5.44120290e-04,
        1.79856130e-05,  2.96318517e-04,  6.54920544e-04,  3.91686683e-04,
       -3.15081303e-04, -4.79838391e-04, -7.06101824e-05, -2.44471110e-04,
       -3.39728246e-04, -1.42441610e-04, -6.58362254e-05, -7.83817456e-05,
       -2.90545366e-04, -2.69784195e-05, -4.68514117e-05,  1.15907284e-04,
       -3.78586052e-05,  1.11022303e-07, -1.24122931e-04,  2.05280237e-04,
       -7.01660952e-05,  1.72084569e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 2821
      nit: 80
     njev: 91
   status: 0
  success: True
        x: array([ 0.49005982, -0.01153472,  0.03800555,  0.04479005, -0.00991089,
        0.06033165,  0.4587844

In [26]:
nodes_to_average = [1058, 412, 670, 1520, 1886, 2418]
local_pos_saved = pd.read_pickle("pickles/2418_clcl.obj")
optimize_mdcm(nodes_to_average, first=False, local_pos=local_pos_saved)

0.5885252836813905
[0.6315854  0.51054029 0.63917093 0.55883842 0.55243    0.6267116 ]
      fun: 0.5243425887379162
 hess_inv: <30x30 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.00142114e-04, -7.82707233e-05,  1.85074178e-04, -5.07371923e-05,
        5.33462163e-04, -7.68385356e-04,  2.66786586e-04, -8.21453994e-04,
        1.21269661e-03,  2.04725120e-04, -4.98601147e-04,  1.14486198e-03,
       -3.76365616e-05,  5.00155473e-04, -7.04658535e-04, -7.29416528e-05,
        5.85975712e-04, -1.15074613e-03,  3.57713859e-04, -2.55462318e-04,
       -9.60342917e-05, -1.37667655e-04,  2.10165219e-04, -4.49640325e-05,
        8.71525051e-05,  1.23678845e-04, -1.56763487e-04, -1.19682042e-04,
        2.41140441e-04, -4.42978987e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 3193
      nit: 90
     njev: 103
   status: 0
  success: True
        x: array([ 0.48423789, -0.01536026,  0.03117761,  0.02500537, -0.00409067,
        0.0619642 ,  0.448463

In [27]:
nodes_to_average = [1886, 72, 412, 462, 567, 660, 1058]
local_pos_saved = pd.read_pickle("pickles/1058_clcl.obj")
optimize_mdcm(nodes_to_average, first=False, local_pos=local_pos_saved)

0.5010281052878824
[0.46849869 0.48203208 0.43307642 0.51426978 0.5200769  0.57151248
 0.5060808 ]
      fun: 0.47637290707246854
 hess_inv: <30x30 LbfgsInvHessProduct with dtype=float64>
      jac: array([-4.44089198e-06, -3.20299343e-05, -2.97539771e-05,  7.73936471e-04,
        4.69624339e-04,  2.52298182e-04, -1.01751937e-04, -1.25455198e-04,
       -2.39031017e-04,  1.57041043e-04, -8.21565016e-05, -5.04929432e-04,
        1.42053040e-04,  5.66213743e-05,  5.65214526e-04, -2.29538610e-04,
        1.57152069e-04,  3.37063701e-04,  2.04836148e-04, -3.70814490e-04,
       -4.24271729e-04, -1.78246307e-04,  1.88793425e-04,  1.67810210e-04,
        2.40696345e-04,  9.69779813e-05, -5.05151462e-05,  1.87849736e-04,
        2.32702746e-04,  1.22069022e-04])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 3069
      nit: 88
     njev: 99
   status: 0
  success: True
        x: array([ 4.75447934e-01, -1.66717854e-02,  2.30698129e-02,  2.19062330e-02,
       -2.6135